<h1 style="color:#520080">Análisis de Homicidios</h1>

<h4 style="color:#946CFF">Importamos librerías</h4>

In [ ]:
import pandas as pd
df = pd.read_csv("database.csv.zip")
#Hemos escogido un CSV muy completo, partiremos con el para hacer las tablas que necesitamos.

C:\Users\rocio\AppData\Local\Temp\ipykernel_4232\2232330604.py:2: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("database.csv.zip")


<h3 style="color:#946CFF">Inspección</h3>

In [3]:
df.head()

,Record ID,Agency Code,Agency Name,Agency Type,City,State,Year,Month,Incident,Crime Type,...,Victim Ethnicity,Perpetrator Sex,Perpetrator Age,Perpetrator Race,Perpetrator Ethnicity,Relationship,Weapon,Victim Count,Perpetrator Count,Record Source
0,1,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,January,1,Murder or Manslaughter,...,Unknown,Male,15,Native American/Alaska Native,Unknown,Acquaintance,Blunt Object,0,0,FBI
1,2,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,March,1,Murder or Manslaughter,...,Unknown,Male,42,White,Unknown,Acquaintance,Strangulation,0,0,FBI
2,3,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,March,2,Murder or Manslaughter,...,Unknown,Unknown,0,Unknown,Unknown,Unknown,Unknown,0,0,FBI
3,4,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,April,1,Murder or Manslaughter,...,Unknown,Male,42,White,Unknown,Acquaintance,Strangulation,0,0,FBI
4,5,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,April,2,Murder or Manslaughter,...,Unknown,Unknown,0,Unknown,Unknown,Unknown,Unknown,0,1,FBI


In [4]:
df.shape

(638454, 24)

In [5]:
df.columns

Index(['Record ID', 'Agency Code', 'Agency Name', 'Agency Type', 'City',
       'State', 'Year', 'Month', 'Incident', 'Crime Type', 'Crime Solved',
       'Victim Sex', 'Victim Age', 'Victim Race', 'Victim Ethnicity',
       'Perpetrator Sex', 'Perpetrator Age', 'Perpetrator Race',
       'Perpetrator Ethnicity', 'Relationship', 'Weapon', 'Victim Count',
       'Perpetrator Count', 'Record Source'],
      dtype='object')

<h3 style="color:#946CFF">División de tablas</h3>

In [11]:
Agencies = df[['Agency Code','Agency Name', 'Agency Type', 'City', 'State' ]]

In [12]:
df['Victim_id'] = 'VIC' + df.index.astype(str)
df['Perpetrator_id'] = 'PER' + df.index.astype(str)
df['Relationship_id'] = 'REL' + df.index.astype(str)

In [ ]:
Incident = df[['Record ID', 'Agency Code', 'Victim_id', 'Perpetrator_id', 'Relationship_id', 'Year', 'Month', 'Crime Type', 'Crime Solved', 'Weapon']]

In [17]:
Victims = df[['Victim_id', 'Record ID', 'Victim Sex', 'Victim Age', 'Victim Race', 'Victim Ethnicity', 'Victim Count']]

In [18]:
Perpetrators = df[['Perpetrator_id', 'Record ID', 'Perpetrator Sex', 'Perpetrator Age', 'Perpetrator Race', 'Perpetrator Ethnicity', 'Perpetrator Count']]

In [19]:
Relationship = df[['Relationship_id', 'Record ID', 'Relationship']]

<h3 style="color:#946CFF">Hypotesis</h3>